In [1]:
import pandas as pd
import numpy as np
import math
import datetime

table = pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table = pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")

def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)

def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

#FIXED tiene conto delle pause pranzo
def difference2(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    
    if(time > datetime.datetime.strptime('12:00', '%H:%M') and 
       time < datetime.datetime.strptime('13:25', '%H:%M') and
       ((time2-time).seconds)/60 > 30):
        time = time + datetime.timedelta(hours=1)
        #print(time)
    return ((time2-time).seconds)/60

In [2]:
def compute(schedule, df):
    row = df[get_row(df, schedule[0]).values]
    tot = 0
    #print(str(schedule[0]) + " " + str(tot))
    for i in range(1, len(schedule.dropna())):
        val = row.loc[:, get_column(df, schedule[i]).values].values[0][0]
        #print(str(schedule[i]) + " " + str(val))
        tot += val
        row = df[get_row(df, schedule[i]).values]
    return round(tot, 2)

def addHeadTail(schedule):
    formatted = list(schedule.dropna())[:]
    formatted.insert(0, "Cremona, Sesto 39")
    formatted.append("Ostiano, Mazzini, 1")
    return formatted

In [3]:
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
fur1_time_greedy = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_time_local = pd.read_csv("../Dati/turni_f/ls1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_time_2opt = pd.read_csv("../Dati/turni_f/2-opt1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_time_base = pd.read_csv("../Dati/turni_f/f1_original.csv", delimiter=";", encoding = "ISO-8859-1")
print("Comparison 1st pickup with greedy, local search and 2opt solutions (Based on time schedule)")
for day in week:
    greedy_schedule = addHeadTail(fur1_time_greedy[day])
    base_schedule = addHeadTail(fur1_time_base[day])
    print(f"Giorno: {day}  "+
          f"{compute(pd.Series(base_schedule), dft)} "+
          f"{compute(pd.Series(greedy_schedule), dft)} "+
          f"{compute(fur1_time_local[day], dft)} "+
          f"{compute(fur1_time_2opt[day], dft)} min")

Comparison 1st pickup with greedy, local search and 2opt solutions (Based on time schedule)
Giorno: lunedi  224.21 195.12 183.97 178.27 min
Giorno: martedi  222.56 260.85 252.93 216.61 min
Giorno: mercoledi  271.43 163.72 163.72 163.72 min
Giorno: giovedi  239.98 246.81 246.06 223.41 min
Giorno: venerdi  235.36 181.04 181.04 170.61 min


In [4]:
fur1_distance_greedy = pd.read_csv("../Dati/turni_f/furgoncino1_KM_routing.csv", delimiter=";")
fur1_distance_local = pd.read_csv("../Dati/turni_f/ls1_km_shifts.csv", delimiter=";")
fur1_distance_2opt = pd.read_csv("../Dati/turni_f/2-opt1_km_shifts.csv", delimiter=";")
print("Comparison 1st pickup with greedy, local search and 2opt solutions (Based on kilometers schedule)")
for day in week:
    greedy_schedule = addHeadTail(fur1_distance_greedy[day])
    print(f"Giorno: {day}  {compute(pd.Series(greedy_schedule), dfd)}  "+
          f"{compute(pd.Series(fur1_distance_local[day]), dfd)}  "+
          f"{compute(pd.Series(fur1_distance_2opt[day]), dfd)} km")

Comparison 1st pickup with greedy, local search and 2opt solutions (Based on kilometers schedule)
Giorno: lunedi  189.52  185.86  161.94 km
Giorno: martedi  314.19  307.48  296.46 km
Giorno: mercoledi  170.99  163.62  155.37 km
Giorno: giovedi  237.83  225.69  186.6 km
Giorno: venerdi  162.72  162.47  159.14 km


In [5]:
def addHeadTail(schedule):
    formatted = list(schedule.dropna())[:]
    formatted.insert(0, "Cremona, Sesto 39")
    formatted.append("San Giovanni in Croce, Feudatari, 25 A")
    return formatted

In [6]:
shortweek = ["martedi", "mercoledi", "giovedi"]
fur2_time_greedy = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")
fur2_time_local = pd.read_csv("../Dati/turni_f/ls2_shifts.csv", delimiter=";")
fur2_time_2opt = pd.read_csv("../Dati/turni_f/2-opt2_shifts.csv", delimiter=";")
fur2_time_base = pd.read_csv("../Dati/turni_f/f2_original.csv", delimiter=";", encoding = "ISO-8859-1")

print("Comparison 2nd pickup with greedy, local search and 2opt solutions (Based on time schedule)")
for day in shortweek:
    greedy_schedule = addHeadTail(fur2_time_greedy[day])
    base_schedule = addHeadTail(fur1_time_base[day])
    print(f"Giorno: {day}  " +
          f"{compute(pd.Series(base_schedule), dft)}  "+
          f"{compute(pd.Series(greedy_schedule), dft)}  "+
          f"{compute(fur2_time_local[day], dft)}  "+
          f"{compute(fur2_time_2opt[day], dft)} min")

Comparison 2nd pickup with greedy, local search and 2opt solutions (Based on time schedule)
Giorno: martedi  201.86  298.71  292.23  237.3 min
Giorno: mercoledi  275.74  143.73  143.73  135.54 min
Giorno: giovedi  243.99  158.78  108.73  108.73 min


In [7]:
fur2_distance_greedy = pd.read_csv("../Dati/turni_f/furgoncino2_KM_routing.csv", delimiter=";")
fur2_distance_local = pd.read_csv("../Dati/turni_f/ls2_km_shifts.csv", delimiter=";")
fur2_distance_2opt = pd.read_csv("../Dati/turni_f/2-opt2_km_shifts.csv", delimiter=";")
print("Comparison 2nd pickup with greedy, local search and 2opt solutions (Based on kilometers schedule)")
for day in shortweek:
    greedy_schedule = addHeadTail(fur2_distance_greedy[day])
    print(f"Giorno: {day}  {compute(pd.Series(greedy_schedule), dfd)}  "+
          f"{compute(pd.Series(fur2_distance_local[day]), dfd)}  "+
          f"{compute(pd.Series(fur2_distance_2opt[day]), dfd)} km")

Comparison 2nd pickup with greedy, local search and 2opt solutions (Based on kilometers schedule)
Giorno: martedi  108.93  108.85  108.67 km
Giorno: mercoledi  164.2  162.67  162.67 km
Giorno: giovedi  236.13  236.13  173.54 km
